# Laboratorium 5 - rekomendacje grafowe

## Przygotowanie

 * dataset i potrzebne biblioteki są dokładnie takie same jak na poprzednim laboratorium
 * pobierz i wypakuj dataset: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
   * więcej możesz poczytać tutaj: https://grouplens.org/datasets/movielens/
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab5`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas sklearn gensim==3.8.3`

## Część 1. - przygotowanie danych

In [1]:
# importujemy wszystkie potrzebne pakiety

import math
import random
import numpy as np
import pandas

from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split, KFold

In [2]:
SCORE_THRESHOLD = 4.0 # recenzje z co najmniej taka ocena wezmiemy pod uwage
VECTOR_SIZE = 20 # jak dlugie powinny byc wektory osadzen wierzcholkow
NEIGHBOURS_WINDOW = 11 # tylu sasiadow wezmiemy pod uwage w algorytmie Word2Vec (symetrycznie i wliczajac biezacy element)
PATH_LENGTH = 30 # dlugosc pojedynczej losowej sciezki
PATHS_COUNT_PER_NODE = 20 # liczba losowych sciezek zaczynajacych sie w kazdym z wierzcholkow

In [4]:
# wczytujemy oceny uytkownikow

ratings = pandas.read_csv('data/ml-latest-small/ratings.csv').drop(columns=['timestamp'])
ratings = ratings.where(ratings['rating'] >= SCORE_THRESHOLD).dropna()
# rozszerzamy ID tak, by sie nie powtarzaly
ratings['userId'] = ratings['userId'].apply(lambda x: 'u_' + str(int(x)))
ratings['movieId'] = ratings['movieId'].apply(lambda x: 'm_' + str(int(x)))
ratings

,userId,movieId,rating
0,u_1,m_1,4.0
1,u_1,m_3,4.0
2,u_1,m_6,4.0
3,u_1,m_47,5.0
4,u_1,m_50,5.0
...,...,...,...
100830,u_610,m_166528,4.0
100831,u_610,m_166534,4.0
100832,u_610,m_168248,5.0
100833,u_610,m_168250,5.0


In [6]:
# wczytujemy gatunki filmow

movies = pandas.read_csv('data/ml-latest-small/movies.csv').drop(columns=['title'])
movies['movieId'] = movies['movieId'].apply(lambda x: 'm_' + str(int(x)))
movies['genres'] = movies['genres'].apply(lambda x: x.split('|'))
movies_to_genres = movies.explode('genres')
movies_to_genres['genres'] = movies_to_genres['genres'].apply(lambda x: 'g_' + x.lower())
movies_to_genres = movies_to_genres.rename(columns = {'genres': 'genre'})
movies_to_genres

,movieId,genre
0,m_1,g_adventure
0,m_1,g_animation
0,m_1,g_children
0,m_1,g_comedy
0,m_1,g_fantasy
...,...,...
9738,m_193583,g_fantasy
9739,m_193585,g_drama
9740,m_193587,g_action
9740,m_193587,g_animation


In [12]:
users = ratings['userId'].unique()
movies = ratings['movieId'].unique()
genres = movies_to_genres['genre'].unique()

['u_1',
 'u_2',
 'u_3',
 'u_4',
 'u_5',
 'u_6',
 'u_7',
 'u_8',
 'u_9',
 'u_10',
 'u_11',
 'u_12',
 'u_13',
 'u_14',
 'u_15',
 'u_16',
 'u_17',
 'u_18',
 'u_19',
 'u_20',
 'u_21',
 'u_22',
 'u_23',
 'u_24',
 'u_25',
 'u_26',
 'u_27',
 'u_28',
 'u_29',
 'u_30',
 'u_31',
 'u_32',
 'u_33',
 'u_34',
 'u_35',
 'u_36',
 'u_37',
 'u_38',
 'u_39',
 'u_40',
 'u_41',
 'u_42',
 'u_43',
 'u_44',
 'u_45',
 'u_46',
 'u_47',
 'u_48',
 'u_49',
 'u_50',
 'u_51',
 'u_52',
 'u_53',
 'u_54',
 'u_55',
 'u_56',
 'u_57',
 'u_58',
 'u_59',
 'u_60',
 'u_61',
 'u_62',
 'u_63',
 'u_64',
 'u_65',
 'u_66',
 'u_67',
 'u_68',
 'u_69',
 'u_70',
 'u_71',
 'u_72',
 'u_73',
 'u_74',
 'u_75',
 'u_76',
 'u_77',
 'u_78',
 'u_79',
 'u_80',
 'u_81',
 'u_82',
 'u_83',
 'u_84',
 'u_85',
 'u_86',
 'u_87',
 'u_88',
 'u_89',
 'u_90',
 'u_91',
 'u_92',
 'u_93',
 'u_94',
 'u_95',
 'u_96',
 'u_97',
 'u_98',
 'u_99',
 'u_100',
 'u_101',
 'u_102',
 'u_103',
 'u_104',
 'u_105',
 'u_106',
 'u_107',
 'u_108',
 'u_109',
 'u_110',
 'u_111'

## Część 2. - spacer po grafie

In [17]:
# generujemy losowe sciezki w grafie
#   krawedzie reprezentowane sa w dwoch macierzach - ratings i movies
#   w wersji podstawowej wszystkie krawedzie traktujemy jako niewazone i nieskierowane
#   mozliwe ulepszenia:
#    - rozwazenie krawedzi skierowanych
#    - uwzglednienie wag krawedzi (ocen uzytkownikow)
#    - jakas forma normalizacji - obnizenia wag wierzcholkow o wysokich stopniach
#    - Node2Vec - parametry P i Q
# wynikiem powinna byc lista list - kazda z tych list zawiera kolejne ID wierzcholkow na sciezce

def node_type(node):
    print(node)
    if node [0] == 'm':
        return 'movieId', 'userId'
    else:
        return 'userId', 'movieId'

def generate_walks(ratings, movies_to_genres, paths_per_node, path_length):
    paths = []
    movies_to_genres['ratings'] = 0.1
    all_edges = pandas.concat([movies_to_genres.rename(columns={'genre':'userId'}), ratings], ignore_index=True)
    for node in list(users) + list(movies) + list(genres):
        for _ in range(paths_per_node):
            path = [node]
            for _ in range(path_length):
                current_type, next_type = node_type(node)
                node_edges = all_edges[all_edges[current_type] == node]
                node = node_edges[next_type].sample()
                path += [node]
            paths += [path]
        
    return paths
    
walks = generate_walks(ratings, movies_to_genres, PATHS_COUNT_PER_NODE, PATH_LENGTH)
print(walks)

u_1
22091    m_151
Name: movieId, dtype: object


KeyError: 0

## Część 3. - obliczenie osadzeń

In [ ]:
# trenujemy model
#   zauwaz, ze wszystkie trzy rodzaje wierzcholkow beda reprezentowane tak samo, w tej samej przestrzeni

model = Word2Vec(sentences=walks, size=VECTOR_SIZE, window=NEIGHBOURS_WINDOW, min_count=1, workers=4)
embeddings = model.wv

## Część 4. - rekomendacje i zastosowania

In [ ]:
PULP_FICTION = 'm_296'
TOY_STORY = 'm_1'
PLANET_OF_THE_APES = 'm_2529'

In [ ]:
# wyszukajmy K najpodobniejszych filmów do danego
# porownaj wyniki dla odleglosci euklidesowej i cosinuswej, np. na trzech powyzszych filmach

def euclidian_distance(i, j):
    pass

def cosine_distance(i, j):
    pass

def k_most_similar_movies(movie_id, K, embeddings, distance_fun):
    # ...
    return k_most_similar

k_most_similar_movies(PULP_FICTION, 5, embeddings, cosine_distance)

In [ ]:
# wyszukajmy k filmow najblizszych uzytkownikowi
# wykorzystaj funkcje z poprzedniej komorki

def k_best_movies_for_user(user_id, K, embeddings, distance_fun):
    # ...
    return k_best_movies

In [ ]:
# sprobujmy czegos bardziej skomplikowanego
#   znajdz ulubiony gatunek filmowy uzytkownika
#   a nastepnie zaproponuj K filmow z tego gatunku - ale nie tych najblizszych uzytkownikowi
#   (zaproponuj, w jaki sposob dobrac filmy interesujace, ale nie z najblizszego otoczenia)

def k_from_favourite_genre(user_id, K, embeddings, distance_fun):
    # ...
    return k_from_genre

In [ ]:
# Na koniec najbardziej skomplikowany algorytm - odpowiednik "Radia utworu" w Spotify.
#   Zaczynamy od jednego filmu, a nastepnie wyznaczamy kolejne, wedrujac po przestrzeni, w ktorej wszystkie elementy sa osadzone.
#   Zaproponuj, jak zdefiniowac podzbior filmow, z ktorych bedziemy wybierac (np. filmy odlegle o min. a i max. b od danego)
#   oraz jak generowac kolejny skok (tak, zeby seria rekomendacji nie byla zbyt monotonna, ale rownoczesnie zgodna z gustem uzytkownika)

def get_playlist(start_movie_id, user_id, K, embeddings):
    # ...
    return playlist